In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from gensim.models import KeyedVectors

Using TensorFlow backend.


In [2]:
try:
    import dill as pickle
except ImportError:
    import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Loading Image Features
train_features = load(open("./drive/My Drive/Project/urdu/urdu_encoded_train_images.pkl", "rb"))
#train_features = load(open("./drive/My Drive/Project/urdu/urdu_encoded_train_images (1).pkl", "rb"))

print('Photos: train=%d' % len(train_features))

Photos: train=500


In [5]:
# Load Train Descriptions, which contains, 5 captions corresponding to a key
train_descriptions = load(open("./drive/My Drive/Project/urdu/urdu_train_descriptions.pkl", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=500


In [6]:
# Create a list of all the training captions
train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        train_captions.append(cap)
len(train_captions)

2495

In [7]:
threshold = 7
word_counts = {}
nsents = 0
for sent in train_captions:
    nsents += 1
    for w in sent.split(' '):
        if w not in word_counts:
            word_counts[w] = 0
        word_counts[w] += 1

vocab = [w for w in word_counts if word_counts[w] >= threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))


preprocessed words 1459 -> 368


In [8]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

369

In [9]:
ixtoword

{1: 'startseq',
 2: 'چھوٹا',
 3: 'لڑکا',
 4: 'سڑک',
 5: 'پر',
 6: 'کھڑا',
 7: 'ہے',
 8: 'جبکہ',
 9: 'آدمی',
 10: 'پتھر',
 11: 'کی',
 12: 'دیوار',
 13: 'کر',
 14: 'رہا',
 15: 'endseq',
 16: 'گلی',
 17: 'میں',
 18: 'ہوئے',
 19: 'بچہ',
 20: 'چل',
 21: 'اور',
 22: 'اس',
 23: 'کے',
 24: 'پیچھے',
 25: 'سفید',
 26: 'قمیض',
 27: 'نیلی',
 28: 'پتلون',
 29: 'مسکراتا',
 30: 'جس',
 31: 'گول',
 32: 'سامنے',
 33: 'بیٹھا',
 34: 'سرخ',
 35: 'رنگ',
 36: 'جیکٹ',
 37: 'ہاکی',
 38: 'کھیلتا',
 39: 'ہوا',
 40: 'کھیل',
 41: 'ہاتھ',
 42: 'لئے',
 43: 'باہر',
 44: 'بیٹھے',
 45: 'کھانا',
 46: 'کھا',
 47: 'کسی',
 48: 'ہوئی',
 49: 'چیز',
 50: 'کو',
 51: 'سے',
 52: 'اپنے',
 53: 'منہ',
 54: 'کچھ',
 55: 'ڈال',
 56: 'میز',
 57: 'اوپر',
 58: 'سبز',
 59: 'پہنے',
 60: 'بہت',
 61: 'دیکھ',
 62: 'دیکھتے',
 63: 'پانی',
 64: 'سی',
 65: 'ہجوم',
 66: 'نوجوان',
 67: 'جرسی',
 68: 'اسکیٹ',
 69: 'بورڈ',
 70: 'چھلانگ',
 71: 'لگاتے',
 72: 'دوسرا',
 73: 'کرتب',
 74: 'دے',
 75: 'ایک',
 76: 'دکھا',
 77: 'اسے',
 78: 'ساتھ',
 79: 'لگا',
 

In [10]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

369

In [11]:
# calculate the length of the description with the most words
def max_length(descriptions): 
    max_l = -1e9
    for key, cap in descriptions.items():
        for line in cap:
            max_l = max(max_l, len(line.split(' ')))
    return max_l

max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 34


In [12]:
urdu_model = KeyedVectors.load_word2vec_format("./drive/My Drive/Project/urdu/urdu_web_news_vector300_word2vec_linguistic_variation_2.bin",binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
urdu_model["نیلے"].shape


(300,)

In [14]:
embedding_dim = 300

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tqdm(wordtoix.items()):
    if word in urdu_model:
        embedding_vector = urdu_model[word]
        embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [15]:
embedding_matrix.shape # (vocab, embedding_dim)

(369, 300)

In [16]:
embedding_matrix[191]

array([-3.94663401e-03, -1.97500899e-01, -3.09703976e-01,  3.36707324e-01,
       -6.63924336e-01,  3.09590660e-02, -4.67106074e-01,  2.35506762e-02,
       -1.59557462e-01, -5.29598534e-01,  2.30298713e-02, -5.24597824e-01,
        7.72665814e-02,  5.58384299e-01, -5.30782223e-01, -1.54872864e-01,
        2.12393537e-01, -3.16560976e-02, -4.83988225e-02,  4.95900571e-01,
       -5.45389295e-01,  2.22988024e-01,  3.03158253e-01, -1.75777629e-01,
       -1.60174891e-01,  2.66699314e-01, -4.51720618e-02, -4.46005970e-01,
       -9.90910921e-04,  6.35023236e-01, -2.49764025e-01, -4.79454398e-01,
        2.31487215e-01,  7.45908245e-02,  2.43936896e-01,  2.28465691e-01,
       -3.71886790e-02,  3.16163301e-02, -1.70750290e-01, -3.29745322e-01,
       -5.52908964e-02,  4.79912609e-01, -4.18980747e-01, -2.09400102e-01,
       -3.02377224e-01, -5.60531497e-01,  4.64844912e-01,  1.45819739e-01,
        3.09847802e-01, -1.93012729e-01,  9.37621519e-02,  2.96744946e-02,
       -9.04227793e-02, -

In [17]:
#with open('theme_Embedding_Matrix.pkl', 'wb') as embedding_pickle:
with open('urdu_Embedding_Matrix.pkl', 'wb') as embedding_pickle:
    pickle.dump(embedding_matrix, embedding_pickle)

In [18]:
#with open('theme_wordtoix.pkl', 'wb') as wti_pickle:
with open('urdu_wordtoix.pkl', 'wb') as wti_pickle:
    pickle.dump(wordtoix, wti_pickle)